# NLTK
___


#### 📦 One-Time Setup (NLTK Resources)

In [ ]:

import nltk
nltk.download( 'punkt' )
nltk.download( 'stopwords' )
nltk.download( 'wordnet' )
nltk.download( 'omw-1.4' )


##### Load Dependencies

In [ ]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer

#### 🧠 Full Pipeline

In [ ]:
# === Preprocessing Configuration ===
EN_STOPWORDS = set( stopwords.words( 'english' ) )
LEMMATIZER = WordNetLemmatizer( )
STEMMER = SnowballStemmer( "english" )

def preprocess_line( line, lower=True, punctuation=True,
                     stopwords=True, lemmatize=True, stem=False ):
    """
    Process a single line of text with optional steps:
    - lower
    - punctuation removal
    - stopword removal
    - lemmatization
    - stemming (optional)
    Returns the cleaned line as a string.
    """
    tokens = word_tokenize( line )
    processed = []
    for token in tokens:
        if lower:
            token = token.lower( )

        if punctuation and token in string.punctuation:
            continue

        if stopwords and token in EN_STOPWORDS:
            continue

        if lemmatize:
            token = LEMMATIZER.lemmatize( token )

        if stem:
            token = STEMMER.stem( token )

        processed.append( token )

    return ' '.join( processed )


def process_file( file_path, **preprocess_kwargs ):
    """
        Read a text file line-by-line, apply preprocessing pipeline to each line,
        and return a list of cleaned lines (original order preserved).
        Pass any keyword args supported by `preprocess_line`.
    """
    cleaned_lines = []
    with open( file_path, 'r', encoding='utf-8' ) as file:
        for line in file:
            cleaned = preprocess_line( line, **preprocess_kwargs )
            cleaned_lines.append( cleaned )
    return cleaned_lines


#### 🔍 Usage

In [ ]:
file_path = 'path/to/Public_Law_118-32.txt'
cleaned_lines = process_file(
    file_path,
    lowercase=True,
    remove_punct=True,
    remove_stopwords=True,
    lemmatize=True,
    stem=False  # Set to True if you want aggressive shortening
)

print( f"Total lines: {len( cleaned_lines )}" )
print( "Example cleaned line:", cleaned_lines[ 0 ] )

## Text Cleaner for PL 118-32

##### Load Dependencies

In [ ]:
import re
import os
from pathlib import Path
import openai
import pandas as pd
import time



#### 1. Load File

In [ ]:
# === 1. Load the Raw Text ===
def load_text( file_path ):
	with open( file_path, 'r', encoding='utf-8' ) as f:
		return f.read( )

#### 2. Clean Document

In [ ]:
def clean_text( text: str ) -> str:
    # Step 1: Normalize newlines
    text = text.replace('\r\n', '\n').replace('\r', '\n')

    # Step 2: Remove page headers and footers (Public Law-specific)
    text = re.sub(r'PUBLIC LAW 118–32.*?\n', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\n\s*\d+\s*\n', '\n', text)  # Remove page numbers between lines

    # Step 3: Remove hyphenation at line breaks (e.g., "appropria-\ntion")
    text = re.sub(r'(\w+)-\n(\w+)', r'\1\2', text)

    # Step 4: Merge broken lines where sentence continues
    text = re.sub(r'(?<!\n)\n(?![\n])', ' ', text)

    # Step 5: Collapse excessive whitespace
    text = re.sub(r'\s+', ' ', text)

    # Step 6: Normalize section markers (optional but useful)
    text = re.sub(r'(SEC\.\s*\d+[A-Z]*\.)', r'\n\n\1', text)  # Add spacing before sections

    return text.strip()


#### 3. Chunk File

In [ ]:
# Simple chunking by words assuming ~1.3 words per token
def chunk_text( text, max_tokens=512 ):
	words = text.split( )
	chunk_size = int( max_tokens * 1.3 )
	chunks = [ ' '.join( words[ i:i + chunk_size ] ) for i in range( 0, len( words ), chunk_size ) ]
	return chunks


##### 🔍 Example

In [ ]:
file_path = 'path_to/Public_Law_118-32.txt'
raw_text = load_text( file_path )
cleaned_text = clean_text( raw_text )
chunks = chunk_text( cleaned_text )
print( f'Total Chunks: {len( chunks )}' )
print( 'Sample chunk:\n', chunks[ 0 ][ :1000 ] )



#  OpenAI Embedding
___

##### API key

In [ ]:
openai.api_key = os.getenv( 'OPENAI_API_KEY' )

#### 1. Define embedding function

In [ ]:
def embed_texts( texts, model='text-embedding-3-small', batch_size=10, sleep=1 ):
    embeddings = []
    for i in range( 0, len( texts ), batch_size ):
        batch = texts[ i:i+batch_size ]
        try:
            response = openai.embeddings.create( input=batch, model=model )
            batch_embeddings = [ e.embedding for e in response.data ]
            embeddings.extend( batch_embeddings )
        except Exception as e:
            print( f'Error at batch {i}: {e}' )
            # Retry or sleep to avoid rate limits
            time.sleep( sleep )
            continue

    return embeddings


#### 2. Embed chunks

In [ ]:
# 2. Embed chunks
embeddings = embed_texts( chunks )

#### 3.  Create DataFrame

In [ ]:
# 3. Create DataFrame
df_embeddings = pd.DataFrame( { chunks, embeddings } )


#### 3. Save


In [ ]:
# 3. Save
df_embeddings.to_parquet( 'public_law_118_32_embeddings.parquet', index=False )


#### 4. Preview

In [ ]:
# 4. Preview
df_embeddings.head(2)

# Text Cleaning
___

#### 1.  Strip irrelevant content  while preserving structure.

In [ ]:
import re

def clean_text( text ):
    # Remove form feeds, line breaks, etc.
    text = re.sub( r'\f+', ' ', text )
    text = re.sub( r'\n+', ' ', text )
    text = re.sub( r'\s{2,}', ' ', text )

    # Normalize Section markers, Title headers, etc.
    text = re.sub( r'SEC\.\s+(\d+)\.', r'Section \1:', text )
    return text.strip( )


#### 2. Chunk the Text

- Large documents need to be chunked (for context window limits during embedding).
- Use semantic or structural chunking.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

seps =[ '\n\n', '\n', '.', ' ' ]
splitter = RecursiveCharacterTextSplitter( chunk_size=1000, chunk_overlap=200, separators=seps )
cleaned = clean_text( raw_text )
chunks = splitter.split_text( cleaned )


#### 3. Generate Embeddings

- Use a language model (e.g., OpenAI, HuggingFace) to create vector representations of each chunk.

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer( 'all-MiniLM-L6-v2' )
embeddings = model.encode( chunks, show_progress_bar=True )


#### 4. Create SQLite Database

- Design a table that links text chunks to their embeddings.

In [ ]:
import sqlite3
import numpy as np
import pickle

conn = sqlite3.connect( 'embeddings.db' )
cursor = conn.cursor( )

cursor.execute('''
CREATE TABLE IF NOT EXISTS law_embeddings (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    chunk TEXT NOT NULL,
    embedding BLOB NOT NULL
)
''')

for chunk, vector in zip( chunks, embeddings ):
    blob = pickle.dumps( vector )
    cursor.execute( 'INSERT INTO law_embeddings (chunk, embedding) VALUES (?, ?)', (chunk, blob) )

conn.commit( )
conn.close( )


####  Retrieval (Vector Search in SQLite)

- You can perform semantic search by encoding a query and comparing via cosine similarity:

In [ ]:
def cosine_similarity( a, b ):
    return np.dot( a, b ) / ( np.linalg.norm( a ) * np.linalg.norm( b ) )

query = 'Appropriations for Department of Defense'
query_vec = model.encode( [ query ] )[ 0 ]

conn = sqlite3.connect( 'embeddings.db' )
cursor = conn.cursor( )
cursor.execute( 'SELECT id, chunk, embedding FROM law_embeddings' )

results = []
for row in cursor.fetchall( ):
    chunk_id, chunk_text, blob = row
    stored_vec = pickle.loads( blob )
    sim = cosine_similarity( query_vec, stored_vec )
    results.append( ( sim, chunk_text ) )

# Sort and get top N
top_matches = sorted( results, key=lambda x: x[ 0 ], reverse=True )[ :5 ]


#   Embedding-Pipeline Script
___

##### Load Dependencies



In [ ]:
import re
import sqlite3
import pickle
import numpy as np
from tqdm import tqdm
import openai
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer

##### Configuration

In [ ]:
# Define paths
TEXT_FILE = 'PublicLaw_118-42.txt'
DB_FILE = 'law_embeddings.db'
EMBEDDING_MODEL = 'all-MiniLM-L6-v2'
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200


##### Load and Clean Raw Text

In [ ]:
def load_and_clean_text( filepath ):
    with open( filepath, 'r', encoding='utf-8' ) as file:
        raw_text = file.read( )

    # Basic normalization
    text = re.sub( r'\f+', ' ', raw_text )
    text = re.sub( r'\n+', ' ', text )
    text = re.sub( r'\s{2,}', ' ', text )
    text = re.sub( r'SEC\.\s+(\d+)\.', r'Section \1:', text )
    return text.strip( )

##### Chunk the Clean Text


In [ ]:
def chunk_text( clean_text ):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP,
        separators=[ '\n\n', '\n', '.', ' ' ]
    )
    return splitter.split_text( clean_text )


##### Generate Embeddings

In [ ]:
def get_embedding( text, model=OPENAI_MODEL ):
    response = openai.Embedding.create(
        input=text,
        model=model
    )
    return response[ 'data' [ 0 ][ 'embedding' ] ]


def embed_chunks( chunks ):
    embeddings = [ ]
    for chunk in tqdm( chunks, desc='Embedding chunks via OpenAI' ):
        try:
            embedding = get_embedding( chunk )
            embeddings.append( embedding )
        except Exception as e:
            print( f'Error embedding chunk: {e}' )
            embeddings.append( [ 0.0 ] * 1536 )  # Placeholder for failed requests
    return embeddings

##### Create SQLite DB



In [ ]:
def create_and_populate_db( chunks, embeddings, db_path ):
    conn = sqlite3.connect( db_path )
    cursor = conn.cursor( )

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS law_embeddings (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        chunk TEXT NOT NULL,
        embedding BLOB NOT NULL
    )
    ''')

    for chunk, vector in zip( chunks, embeddings ):
        blob = pickle.dumps( vector )
        cursor.execute(
            'INSERT INTO law_embeddings ( chunk, embedding ) VALUES ( ?, ? )',
            ( chunk, blob )
        )

    conn.commit( )
    conn.close( )


#### Script

In [ ]:
# === MAIN ===
def main():
    print('Step 1: Load and clean text')
    cleaned_text = load_and_clean_text(TEXT_FILE)

    print('Step 2: Chunking text')
    chunks = chunk_text(cleaned_text)
    print(f'Total chunks: {len(chunks)}')

    print('Step 3: Embedding with OpenAI API')
    embeddings = embed_chunks(chunks)

    print('Step 4: Saving to SQLite')
    create_and_populate_db(chunks, embeddings, DB_FILE)

    print(f'Pipeline complete. Embeddings stored in: {DB_FILE}')


if __name__ == '__main__':
    main()

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd

# === 1. Load Model ===
# You can try other models like 'all-MiniLM-L6-v2', 'all-mpnet-base-v2', or 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer( 'all-MiniLM-L6-v2' )

# === 2. Embed Chunks ===
def embed_with_sentence_transformers( texts, model ):
    return model.encode( texts, show_progress_bar=True, convert_to_numpy=True )

local_embeddings = embed_with_sentence_transformers( chunks, model )

# === 3. Save in a DataFrame ===
df_local = pd.DataFrame({
    'chunk': chunks,
    'embedding': list( local_embeddings )  # numpy arrays to list for DataFrame compatibility
})

# === 4. Save to Disk ===
df_local.to_parquet( 'public_law_118_32_local_embeddings.parquet', index=False )

# === 5. Preview ===
df_local.head( 2 )




### 🔍 Example Usage

